<center>
<img src="https://supportvectors.ai/logo-poster-transparent.png" width="400px" style="opacity:0.7">
</center>

In [1]:
%run supportvectors-common.ipynb


<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



# Using Instructor Framework

## Instructor Framework for LLMs: OpenAI and Ollama



Instructor is a Python framework that enhances OpenAI's API and similar LLMs by enforcing structured output in the form of Pydantic models.

Install Instructor by bringing in the following libraries (already included as part of `svlearn-bootcamp`):

`uv add instructor pydantic openai`


In [2]:
# Imports includes patch from the instructor framework

from instructor import patch
from openai import OpenAI
from pydantic import BaseModel
from rich import print as rprint



### 1. Connecting to OpenAI API with Instructor


In [3]:

# First, create a patched OpenAI client (your OPENAI_API_KEY should be in the .env file)
client = patch(OpenAI())


In [ ]:
from pydantic import Field
# Define a structured output model
class ResponseModel(BaseModel):
    summary: str = Field(description="A concise summary of the input text.")
    keywords: list[str] = Field(description="A list of keywords extracted from the input text.")


In [5]:

# Query OpenAI with structured response enforcement
response = client.chat.completions.create(
    model="gpt-4o-mini", #"gpt-4-turbo",
    messages=[{"role": "user", "content": "Summarize the theory of relativity and provide keywords."}],
    response_model=ResponseModel  # Enforces structured response
)


In [6]:
rprint(response)

ResponseModel(
    summary='The theory of relativity, developed by Albert Einstein, consists of two parts: special relativity and 
general relativity. Special relativity, introduced in 1905, postulates that the laws of physics are the same for 
all non-accelerating observers and establishes that the speed of light is constant in a vacuum, which leads to the 
conclusion that space and time are interwoven into a single continuum known as spacetime. General relativity, 
published in 1915, extends this concept by describing gravitation as the curvature of spacetime caused by mass. 
This implies that massive objects warp the fabric of space and time, affecting the motion of other objects. The 
theory has been confirmed by numerous experimental observations and is foundational to modern physics, influencing 
cosmology, astrophysics, and our understanding of the universe.',
    keywords=[
        'Einstein',
        'theory of relativity',
        'special relativity',
        'general relativity',
        'spacetime',
        'gravity',
        'mass',
        'curvature',
        'physics',
        'cosmology',
        'astrophysics'
    ]
)


### 2. Using Instructor with a Locally Hosted Ollama Model


In [7]:
import instructor
# To use Ollama, ensure it is running locally
# We need to set base_url to the local Ollama API endpoint
ollama_client = patch(OpenAI(base_url="http://localhost:11434/v1/"),
                      mode=instructor.Mode.JSON,)


In [8]:

# Request structured response from a local LLM (e.g., Mistral or Llama3)
response = ollama_client.chat.completions.create(
    model= "mistral",#"llama3.2",#
    messages=[{"role": "user", "content": "Explain quantum entanglement and list key principles."}],
    response_model=ResponseModel,
)


In [9]:

rprint(response)


ResponseModel(
    summary='Quantum Entanglement is a phenomenon in quantum physics where two or more particles become 
interconnected and the state of one particle can instantly affect the state of the other, regardless of the 
distance between them. This connection persists even when the particles are separated by vast distances.',
    keywords=[
        'quantum physics',
        'entanglement',
        'superposition',
        'non-locality',
        'Einstein-Podolsky-Rosen (EPR) paradox',
        'Bell inequality',
        'Quantum Entangled states'
    ]
)


### 3. Handling More Complex Outputs with Pydantic Models


In [16]:

# Define a more complex structured model
class DetailedResponseModel(BaseModel):
    summary: str
    keywords: list[str]
    references: list[str]


In [17]:

# Query OpenAI with a detailed structured response
response = client.chat.completions.create(
    model="gpt-4o-mini", #"gpt-4-turbo",
    messages=[{"role": "user", "content": "Describe black holes and list references for further reading."}],
    response_model=DetailedResponseModel
)


In [18]:

rprint(response)


DetailedResponseModel(
    summary='Black holes are regions in space where the gravitational pull is so strong that nothing, not even 
light, can escape from them. They are formed when massive stars collapse under their own gravity at the end of 
their life cycle. The boundary surrounding a black hole is known as the event horizon, beyond which no information 
or matter can return. Black holes can be categorized into different types, including stellar black holes, 
supermassive black holes, and intermediate black holes, based on their mass. They play a crucial role in the 
structure and evolution of galaxies.',
    keywords=[
        'black holes',
        'event horizon',
        'stellar black holes',
        'supermassive black holes',
        'gravity',
        'astronomy'
    ],
    references=[
        '"Black Holes and Time Warps: Einstein\'s Outrageous Legacy" by Kip S. Thorne',
        '"The Black Hole War: My Battle with Stephen Hawking to Make the World Safe for Quantum Mechanics" by 
Leonard Susskind',
        '"A Brief History of Time" by Stephen Hawking',
        'NASA: Black Holes - https://www.nasa.gov/blackholes',
        '"The Mathematical Theory of Black Holes" by S. Chandrasekhar'
    ]
)


### 4. Customizing Behavior with Instructor


In [13]:
from pydantic import field_validator
# Instructor allows for setting validation rules and customizing parsing behavior.
# For example, ensuring output lists a minimum number of keywords:
class StrictResponseModel(BaseModel):
    summary: str
    keywords: list[str]
    references: list[str]
    
    @field_validator("keywords")
    @classmethod
    def check_keywords_length(cls, v):
        if len(v) < 3:
            raise ValueError("At least 3 keywords required.")
        return v


In [14]:

response = client.chat.completions.create(
    model="gpt-4o-mini", #"gpt-4-turbo",
    messages=[{"role": "user", "content": "Explain superconductivity with key takeaways."}],
    response_model=StrictResponseModel
)


In [15]:

rprint(response)


StrictResponseModel(
    summary='Superconductivity is a phenomenon observed in certain materials where they exhibit zero electrical 
resistance and the expulsion of magnetic fields when cooled below a characteristic critical temperature. This leads
to the potential for highly efficient power transmission, advanced magnetic levitation, and significant 
applications in various fields including medical imaging and particle accelerators.',
    keywords=[
        'superconductivity',
        'zero electrical resistance',
        'expulsion of magnetic fields',
        'critical temperature',
        'applications'
    ],
    references=[
        'John Bardeen, Leon Cooper, and Robert Schrieffer, "Theory of Superconductivity", Physical Review, 1957',
        'Paul M. Chaikin and Troy C. Lubensky, "Principles of Condensed Matter Physics", Cambridge University 
Press, 1995',
        'Markus P. E. D. M. Karpinski, "Superconductivity: A Very Short Introduction", Oxford University Press, 
2018'
    ]
)


### 5. Summary
- Instructor helps enforce structured LLM output using Pydantic.
- It works with both OpenAI's API and local models via Ollama.
- Using Pydantic, we can enforce validation, typing, and custom logic.

This notebook demonstrates practical usage of Instructor for structured AI output.
